## Kaggle competiotion
## IMDB Dataset of 50K Movie Reviews

https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [2]:
import os
import json
import kaggle.api

#from fastbook import *
from fastai.text.all import *

import zipfile # to extract data from zip archive

#### Data Block Tutorial   https://docs.fast.ai/tutorial.datablock

#### Torchtext example   https://towardsdatascience.com/deep-learning-for-nlp-with-pytorch-and-torchtext-4f92d69052f

In [3]:
os.getcwd()

'/jet/prs/workspace/Kaggle_IMDB'

In [3]:
api_token = '{"username":"fornitroll","key":"f89ce16bfe131d4f027f39748aac27ed"}'

In [4]:
# create the kaggle folder in the current work directory
!mkdir /jet/prs/workspace/Kaggle_IMDB/kaggle

mkdir: cannot create directory ‘/jet/prs/workspace/Kaggle_IMDB/kaggle’: File exists


In [5]:
# create the file kaggle.json in the folder
!touch /jet/prs/workspace/Kaggle_IMDB/kaggle/kaggle.json

In [6]:
with open('/jet/prs/workspace/Kaggle_IMDB/kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

In [7]:
# restrict the access to the file in kaggle folder
!chmod 600 /kaggle

chmod: cannot access '/kaggle': No such file or directory


In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 97%|████████████████████████████████████▉ | 25.0M/25.7M [00:00<00:00, 31.7MB/s]
100%|██████████████████████████████████████| 25.7M/25.7M [00:00<00:00, 81.9MB/s]


### Extract the file

In [14]:
#file_extract??
# extract the file
zipfile.ZipFile('imdb-dataset-of-50k-movie-reviews.zip').extractall()

In [4]:
path = Path('dataset')

In [5]:
path/'IMDB Dataset.csv'

Path('dataset/IMDB Dataset.csv')

In [6]:
df = pd.read_csv(path/'IMDB Dataset.csv')

In [7]:
df.head(2)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses main...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his ...",positive


In [8]:
# use only 2000 examples to speed-up trainig
df = df.iloc[:2000]

In [9]:
# here is fastai 2 , fastai 1 has another code (use DataBunch)
# by default fastai use splitter=ColReader(), by the column is_valid, we use RandomSplitter herre -- by def valid set=0.2
imdb_lm = DataBlock(blocks = TextBlock.from_df('review', is_lm=True),
                   get_x=ColReader('text'),
                   splitter=RandomSplitter(seed=42))
dls = imdb_lm.dataloaders(df)

/jet/var/python/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [9]:
dls.show_batch(max_n=3)

,text,text_
0,"xxbos i think i watched this movie , but do n't quote me , as i may have fallen asleep during watching it as it did n't exactly "" grip my excitement and imagination . "" xxmaj at least i know i watched enough of it to know i wo n't be watching it again soon . xxmaj or ever . \n\n xxmaj jeez , talk about lame … xxmaj really lame","i think i watched this movie , but do n't quote me , as i may have fallen asleep during watching it as it did n't exactly "" grip my excitement and imagination . "" xxmaj at least i know i watched enough of it to know i wo n't be watching it again soon . xxmaj or ever . \n\n xxmaj jeez , talk about lame … xxmaj really lame ."
1,that with all the slapstick moments.the movie show 's some moments of how the xxmaj red xxmaj sox nation ( in xxmaj fenway xxmaj xxunk the fans felt about 86 years of the xxmaj sox always screwed up at the end of the season and how the love of the xxmaj sox and the love with another human go hand to hand . xxbos xxmaj this unsung quiet gem tells the true,with all the slapstick moments.the movie show 's some moments of how the xxmaj red xxmaj sox nation ( in xxmaj fenway xxmaj xxunk the fans felt about 86 years of the xxmaj sox always screwed up at the end of the season and how the love of the xxmaj sox and the love with another human go hand to hand . xxbos xxmaj this unsung quiet gem tells the true story
2,"heero and xxmaj trowa . \n\n xxmaj relena xxmaj xxunk sounds older in xxmaj english , voiced by xxmaj lisa xxmaj ann xxmaj bailey . xxmaj this might not sit well with her youthful personification early in the series but as her character matures later into the story , her voice follows suit and ends up fitting in very well with the character development . \n\n xxmaj zechs xxmaj merquise would be","and xxmaj trowa . \n\n xxmaj relena xxmaj xxunk sounds older in xxmaj english , voiced by xxmaj lisa xxmaj ann xxmaj bailey . xxmaj this might not sit well with her youthful personification early in the series but as her character matures later into the story , her voice follows suit and ends up fitting in very well with the character development . \n\n xxmaj zechs xxmaj merquise would be one"


In [10]:
dls.vocab[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the']

### Fine-Tuning language model

In [12]:
learn = language_model_learner(
    dls, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()])

In [13]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.266078,4.015866,0.277464,55.471302,05:13


In [62]:
#learn.save('after_fit_one_cycle')

In [63]:
#learn = learn.load('after_fit_one_cycle')

In [14]:
learn.unfreeze()
learn.fit_one_cycle(2, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.881929,3.962972,0.282223,52.613457,06:57
1,3.593167,3.940696,0.287117,51.454376,06:59


In [ ]:
learn.save_encoder('finetuned')

### Create Classifier base on the fine tuned language model

In [15]:
imdb_clas = DataBlock(blocks=(TextBlock.from_df('review', seq_len=72, vocab=dls.vocab), CategoryBlock),
                      get_x=ColReader('text'),
                      get_y=ColReader('sentiment'),
                      splitter=RandomSplitter(seed=42))

In [16]:
dls = imdb_clas.dataloaders(df, bs=64, seq_len=72)

/jet/var/python/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [17]:
dls.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj warning : xxmaj does contain spoilers . \n\n xxmaj open xxmaj your xxmaj eyes \n\n xxmaj if you have not seen this film and plan on doing so , just stop reading here and take my word for it . xxmaj you have to see this film . i have seen it four times so far and i still have n't made up my mind as to what exactly happened in the film . xxmaj that is all i am going to say because if you have not seen this film , then stop reading right now . \n\n xxmaj if you are still reading then i am going to pose some questions to you and maybe if anyone has any answers you can email me and let me know what you think . \n\n i remember my xxmaj grade 11 xxmaj english teacher quite well . xxmaj",positive
1,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,positive
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,negative


In [18]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy)

In [ ]:
learn = learn.load_encoder('finetuned')

In [19]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.675456,0.579439,0.727500,02:28


In [20]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.604507,0.452863,0.815000,02:49


In [21]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.472115,0.385056,0.820000,04:24


In [22]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.352967,0.329699,0.867500,06:56
1,0.314443,0.327349,0.872500,06:33


In [23]:
learn.predict("I really liked that movie!")

('positive', tensor(1), tensor([0.3607, 0.6393]))

In [29]:
# take the unseen review for prediction
df_test = pd.read_csv(path/'IMDB Dataset.csv')
df_test = df_test.iloc[2000:]
#rev = 

In [47]:
learn.predict(df_test.review[2001])[0]== df_test.sentiment[2001]

False

In [58]:
# get the predictions for unseen data, take only 1000 samples
s = 0
for i in range(2000,1000+2000):
    if learn.predict(df_test.review[i])[0]==df_test.sentiment[i]:
        s+=1

In [60]:
# calculate the accuracy for unseen data, take only 1000 samples
acc = s/(1000)
acc

0.853

In [61]:
learn.save('final_model_cls')

Path('models/final_model_cls.pth')